In [4]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

In [5]:
from PIL import Image
import cv2

In [6]:
import os
import sys

In [57]:
from math import floor, ceil

In [79]:
from tensorflow import convert_to_tensor, Tensor, constant

In [15]:
os.path.isdir('./dataset')

True

In [43]:
imgpaths = {}

In [44]:
pth = './dataset/png'
for d in os.listdir(pth):
    if not os.path.isdir(pth + '/' + d):
        continue
    for img in os.listdir(pth + '/' +d):
        path = './dataset/png/'+d
        if d not in imgpaths:
            imgpaths[d] = []
        imgpaths[d].append(path + '/' + img)

In [45]:
imgpaths.keys()

dict_keys(['airplane', 'alarm clock', 'angel', 'ant', 'apple', 'arm', 'armchair', 'ashtray', 'axe', 'backpack', 'banana', 'barn', 'baseball bat', 'basket', 'bathtub', 'bear (animal)', 'bed', 'bee', 'beer-mug', 'bell', 'bench', 'bicycle', 'binoculars', 'blimp', 'book', 'bookshelf', 'boomerang', 'bottle opener', 'bowl', 'brain', 'bread', 'bridge', 'bulldozer', 'bus', 'bush', 'butterfly', 'cabinet', 'cactus', 'cake', 'calculator', 'camel', 'camera', 'candle', 'cannon', 'canoe', 'car (sedan)', 'carrot', 'castle', 'cat', 'cell phone', 'chair', 'chandelier', 'church', 'cigarette', 'cloud', 'comb', 'computer monitor', 'computer-mouse', 'couch', 'cow', 'crab', 'crane (machine)', 'crocodile', 'crown', 'cup', 'diamond', 'dog', 'dolphin', 'donut', 'door', 'door handle', 'dragon', 'duck', 'ear', 'elephant', 'envelope', 'eye', 'eyeglasses', 'face', 'fan', 'feather', 'fire hydrant', 'fish', 'flashlight', 'floor lamp', 'flower with stem', 'flying bird', 'flying saucer', 'foot', 'fork', 'frog', 'fryin

In [47]:
imgpaths['airplane'][:10]

['./dataset/png/airplane/1.png',
 './dataset/png/airplane/10.png',
 './dataset/png/airplane/11.png',
 './dataset/png/airplane/12.png',
 './dataset/png/airplane/13.png',
 './dataset/png/airplane/14.png',
 './dataset/png/airplane/15.png',
 './dataset/png/airplane/16.png',
 './dataset/png/airplane/17.png',
 './dataset/png/airplane/18.png']

In [8]:
os.listdir('./dataset/png')

['airplane',
 'alarm clock',
 'angel',
 'ant',
 'apple',
 'arm',
 'armchair',
 'ashtray',
 'axe',
 'backpack',
 'banana',
 'barn',
 'baseball bat',
 'basket',
 'bathtub',
 'bear (animal)',
 'bed',
 'bee',
 'beer-mug',
 'bell',
 'bench',
 'bicycle',
 'binoculars',
 'blimp',
 'book',
 'bookshelf',
 'boomerang',
 'bottle opener',
 'bowl',
 'brain',
 'bread',
 'bridge',
 'bulldozer',
 'bus',
 'bush',
 'butterfly',
 'cabinet',
 'cactus',
 'cake',
 'calculator',
 'camel',
 'camera',
 'candle',
 'cannon',
 'canoe',
 'car (sedan)',
 'carrot',
 'castle',
 'cat',
 'cell phone',
 'chair',
 'chandelier',
 'church',
 'cigarette',
 'cloud',
 'comb',
 'computer monitor',
 'computer-mouse',
 'couch',
 'cow',
 'crab',
 'crane (machine)',
 'crocodile',
 'crown',
 'cup',
 'diamond',
 'dog',
 'dolphin',
 'donut',
 'door',
 'door handle',
 'dragon',
 'duck',
 'ear',
 'elephant',
 'envelope',
 'eye',
 'eyeglasses',
 'face',
 'fan',
 'feather',
 'filelist.txt',
 'fire hydrant',
 'fish',
 'flashlight',
 'floo

In [84]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, db_dir, batch_size, input_shape, num_classes=None, 
                 shuffle=True, dir_classes=False):
        
        # todo: fix num classes
        
        self.current_counter = 0
        self.label_dict = {}

        self.input_shape = input_shape
        self.batch_size = batch_size
        self.num_classes = num_classes
        self.shuffle = shuffle
        
        # load the data from the root directory
        self.data, self.labels = self.get_data(db_dir)
        self.indices = np.arange(len(self.data))
        self.on_epoch_end()


    def get_data(self, root_dir):
        """"
        Loads the paths to the images and their corresponding labels from the database directory
        """
        self.data = []
        self.labels = []

        path = root_dir
        pth = path
        
        #pth = './dataset/png'
        for d in os.listdir(pth):
            if not os.path.isdir(pth + '/' + d):
                continue
            for img in os.listdir(pth + '/' + d):
                path = './dataset/png/'+ d
                if d not in imgpaths:
                    imgpaths[d] = []
                imgpaths[d].append(path + '/' + img)
                
        for i in imgpaths.keys():
            for j in imgpaths[i]:
                self.data.append(j)
                self.labels.append(i)

        return self.data, self.labels

    def __len__(self):
        """
        Returns the number of batches per epoch: the total size of the dataset divided by the batch size
        """
        return int(np.floor(len(self.data) / self.batch_size))
        # TODO your code  here (size of dataset divided by the batch size)
        return 0
    
    def vec_to_tensor(self, v):
      return constant(v)
    
    """
        Preprocesses before outputting to .fit()
        This can mean: reshape, channel count check, mean subtraction and std. dev division.
    """
    def batch_to_train(self, batch_x, batch_y):
        x_train = [self.vec_to_tensor(x) for x in batch_x]
        y_train = [self.vec_to_tensor(x) for x in batch_y]

        #print(self.input_shape)
        x_train = [cv2.resize(np.array(i), (self.input_shape[0], self.input_shape[1])) for i in x_train]
        x_train2 = []
        y_train2 = []

        for i in range(len(x_train)):
          if (x_train[i].shape[0] == self.input_shape[0] and x_train[i].shape[1] == self.input_shape[1]):
            x_train2.append(x_train[i])
            y_train2.append([y_train[i].numpy(), 0 if y_train[i].numpy() == 1 else 1])

        y_test = y_train2[:floor(1/5 * self.batch_size)]
        x_test = x_train2[:floor(1/5 * self.batch_size)]

        x_train = x_train2[ceil(1/5 * self.batch_size + 1):]
        y_train = y_train2[ceil(1/5 * self.batch_size + 1):]

        
        
        x_train = np.array(x_train)
        y_train = np.array(y_train)
        
#         mean = x_train.mean(axis=(0, 1, 2))
#         x_train[..., 0] = x_train[..., 0] - mean[0]
#         x_train[..., 1] = x_train[..., 1] - mean[1]
#         x_train[..., 2] = x_train[..., 2] - mean[2]
        
        x_test = np.array(x_test)
        y_test = np.array(y_test)
            
        return x_train, y_train

    def __getitem__(self, index):
        """"
        Generates a batch of data
        Note: computation intensivity
        """

        batch_indices = self.indices[index*self.batch_size : (index+1)*self.batch_size]
        
        batch_x = [] # TODO load the image(s) from batch_indices
        batch_y = [] # TODO load the corresponding labels of the images you loaded

        for i in batch_indices:
          ci = self.data[i]
          cl = self.labels[i]
            
          cd = Image.open(ci)
          cd = np.asarray(cd)

#           if (len(cd.shape) != 3):
#             continue
          # reshape?

          batch_x.append(cd)
          batch_y.append(cl)
        
         

        # optionally you can use: batch_y = tf.keras.utils.to_categorical(batch_y, num_classes=self.num_classes)
        return self.batch_to_train(batch_x, batch_y)

    def on_epoch_end(self):
        """"
        Called at the end of each epoch
        """
        # if required, shuffle your data after each epoch
        self.indices = np.arange(len(self.data))
        if self.shuffle:
            # TODO shuffle data
            # you might find np.random.shuffle useful here
            np.random.shuffle(self.indices)


In [85]:
d = DataGenerator(
    db_dir='./dataset/png',
    batch_size=4,
    input_shape=(120, 120),
    dir_classes=True
)

In [86]:
d.data

['./dataset/png/airplane/1.png',
 './dataset/png/airplane/10.png',
 './dataset/png/airplane/11.png',
 './dataset/png/airplane/12.png',
 './dataset/png/airplane/13.png',
 './dataset/png/airplane/14.png',
 './dataset/png/airplane/15.png',
 './dataset/png/airplane/16.png',
 './dataset/png/airplane/17.png',
 './dataset/png/airplane/18.png',
 './dataset/png/airplane/19.png',
 './dataset/png/airplane/2.png',
 './dataset/png/airplane/20.png',
 './dataset/png/airplane/21.png',
 './dataset/png/airplane/22.png',
 './dataset/png/airplane/23.png',
 './dataset/png/airplane/24.png',
 './dataset/png/airplane/25.png',
 './dataset/png/airplane/26.png',
 './dataset/png/airplane/27.png',
 './dataset/png/airplane/28.png',
 './dataset/png/airplane/29.png',
 './dataset/png/airplane/3.png',
 './dataset/png/airplane/30.png',
 './dataset/png/airplane/31.png',
 './dataset/png/airplane/32.png',
 './dataset/png/airplane/33.png',
 './dataset/png/airplane/34.png',
 './dataset/png/airplane/35.png',
 './dataset/png/a

In [87]:
d[0]

(array([[[255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         ...,
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255]],
 
        [[255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         ...,
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255],
         [255, 255, 255, ..., 255, 255, 255]]], dtype=uint8),
 array([[b'mug', b'1'],
        [b'mug', b'1']], dtype='|S11'))

In [91]:
d[0][0][0][1]

array([255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255, 255,
       255, 255, 255], dtype=uint8)